In [1]:
import os
import sys
import tqdm
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
class Config:
    ROOT_DIR = './data/huge'
    SEED = 42
    
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    BATCH_SIZE = 64
    EPOCHS = 100
    LEARNING_RATE = 0.001
    SERIES_LENGTH = 50

CONFIG = Config()

In [3]:
seed = CONFIG.SEED
np.random.seed(seed)
torch.manual_seed(seed)

In [4]:
train_df = pd.read_csv(os.path.join(CONFIG.ROOT_DIR, "train_df.csv"), index_col='Serial Number')
train_y = pd.read_csv(os.path.join(CONFIG.ROOT_DIR, "train_y.csv"), index_col='Serial Number')
pred_x = pd.read_csv(os.path.join(CONFIG.ROOT_DIR, "pred_x.csv"), index_col='Serial Number')

In [5]:
train_df['TIMESTAMP'] = pd.to_datetime(train_df['TIMESTAMP'])
train_df['TIMESTAMP'] = train_df['TIMESTAMP'].map(lambda t: t.strftime('%Y-%m-%d %H:%M'))

pred_x['TIMESTAMP'] = pd.to_datetime(pred_x['TIMESTAMP'])
pred_x['TIMESTAMP'] = pred_x['TIMESTAMP'].map(lambda t: t.strftime('%Y-%m-%d %H:%M'))

serial_key = list(train_df.columns)[0]
date_time_key = list(train_df.columns)[1]
feature_keys = list(train_df.columns)[3:-1]
target_key = list(train_df.columns)[-1]

train_x = train_df.drop(columns='Y')

In [6]:
print("Train Data 크기 :", train_x.shape)
print("Pred Data 크기 :", pred_x.shape)

print("Train Data의 Serial Number의 unique 값 :", len(train_x[serial_key].unique()))
print("Pred Data의 Serial Number의 unique 값 :", len(pred_x[serial_key].unique()))

Train Data 크기 : (555456, 19)
Pred Data 크기 : (138880, 19)
Train Data의 Serial Number의 unique 값 : 97860
Pred Data의 Serial Number의 unique 값 : 72562


In [7]:
train_x.head(3)

,TIMESTAMP,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
Serial Number,,,,,,,,,,,,,,,,,,,
19,2020-02-09 16:24,EI83N072710203N8H,22,18,22,16,32,99,91219,43095,2132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,2020-02-10 16:39,EI83N072710203N8H,22,18,22,16,32,99,91413,43214,2133,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,2020-02-11 16:54,EI83N072710203N8H,22,18,22,16,32,99,91606,43332,2135,49.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


### PreProcessing

In [8]:
train_x.drop(columns='X1', inplace=True)
pred_x.drop(columns='X1', inplace=True)

In [9]:
scaler = StandardScaler()

train_x[feature_keys] = scaler.fit_transform(train_df[feature_keys])
pred_x[feature_keys] = scaler.transform(pred_x[feature_keys])

In [10]:
train_x_by_serial = [group[1] for group in train_x.groupby(train_x.index)]
pred_x_by_serial = [group[1] for group in pred_x.groupby(pred_x.index)]

train_x_by_serial = [group.sort_values('TIMESTAMP') for group in train_x_by_serial]
pred_x_by_serial = [group.sort_values('TIMESTAMP') for group in pred_x_by_serial]

### Dataset

In [11]:
def train_val_test_split(Xs, ys, val_ratio=0.2, test_ratio=0.1):
    data_per_label = {}

    for x, y in zip(Xs, ys):
        label = y
        if label not in data_per_label:
            data_per_label[label] = []
        data_per_label[label].append((x, y))

    train, val, test = [], [], []

    for label in data_per_label:
        data = data_per_label[label]
        n_test = int(len(data) * test_ratio)
        n_val = int(len(data) * val_ratio)
        test += data[:n_test]
        val += data[n_test:n_test+n_val]
        train += data[n_test+n_val:]

    X_train, y_train = zip(*train)
    X_val, y_val = zip(*val)
    X_test, y_test = zip(*test)

    return X_train, X_val, X_test, y_train, y_val, y_test

In [12]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(train_x_by_serial, train_y['Y'])

X_train = [x.drop(columns='TIMESTAMP') for x in X_train]
X_val = [x.drop(columns='TIMESTAMP') for x in X_val]
X_test = [x.drop(columns='TIMESTAMP') for x in X_test]

print("Train Data의 개수 :", len(X_train))
print("Validation Data의 개수 :", len(X_val))
print("Test Data의 개수 :", len(X_test))

Train Data의 개수 : 5791
Validation Data의 개수 : 1654
Test Data의 개수 : 827


In [13]:
def align_data(data, series_length):
    data_features = [x[feature_keys] for x in data]
    length_aligned_X = []
    
    for x in data_features:
        if len(x) >= series_length:
            length_aligned_X.append(x[-series_length:])
        else:
            length_aligned_X.append(x.append([x.iloc[-1]] * (series_length - len(x))))
    return np.array(length_aligned_X)

In [14]:
series_length = 50

X_train = align_data(X_train, series_length)
X_val = align_data(X_val, series_length)
X_test = align_data(X_test, series_length)

In [15]:
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

### Dataset

In [16]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.float32)

In [17]:
train_dataset = TimeSeriesDataset(X_train, y_train)
val_dataset = TimeSeriesDataset(X_val, y_val)
test_dataset = TimeSeriesDataset(X_test, y_test)

In [18]:
train_loader = DataLoader(train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)

### Model

In [19]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), hidden_dim).to(x.device)
        c0 = torch.zeros(1, x.size(0), hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

In [20]:
input_dim = len(feature_keys)
hidden_dim = 50
output_dim = 1

In [21]:
model = LSTMModel(input_dim, hidden_dim, output_dim)
model = model.to(CONFIG.DEVICE)

In [22]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=CONFIG.LEARNING_RATE)

### Training

In [23]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    device = CONFIG.DEVICE
    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        
        val_loss = 0.0
        model.eval()
        all_preds = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), labels)
                val_loss += loss.item() * inputs.size(0)
                all_preds.extend(outputs.squeeze().cpu().numpy())
        
        train_loss /= len(train_loader.dataset)
        val_loss /= len(val_loader.dataset)
        
        val_preds = (np.array(all_preds) > 0.5).astype(int)
        f1 = f1_score(y_val, val_preds, average='macro')
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val F1 Score: {f1:.4f}')
    
    return model

In [24]:
model = train_model(model, train_loader, val_loader, criterion, optimizer, CONFIG.EPOCHS)

Epoch 1/100, Train Loss: 0.2903, Val Loss: 0.1759, Val F1 Score: 0.8563
Epoch 2/100, Train Loss: 0.1078, Val Loss: 0.1219, Val F1 Score: 0.8990
Epoch 3/100, Train Loss: 0.0953, Val Loss: 0.1262, Val F1 Score: 0.8939
Epoch 4/100, Train Loss: 0.0840, Val Loss: 0.0984, Val F1 Score: 0.9159
Epoch 5/100, Train Loss: 0.0772, Val Loss: 0.0922, Val F1 Score: 0.9292
Epoch 6/100, Train Loss: 0.0739, Val Loss: 0.0921, Val F1 Score: 0.9306
Epoch 7/100, Train Loss: 0.0693, Val Loss: 0.0930, Val F1 Score: 0.9266
Epoch 8/100, Train Loss: 0.0755, Val Loss: 0.1401, Val F1 Score: 0.8732


KeyboardInterrupt: 

In [52]:
model.eval()
test_preds = []

with torch.no_grad():
    for inputs, masks, deltas, labels in test_loader:
        inputs, masks, deltas = inputs.to(CONFIG.DEVICE), masks.to(CONFIG.DEVICE), deltas.to(CONFIG.DEVICE)
        outputs = model(inputs, masks, deltas)
        test_preds.extend(outputs.squeeze().cpu().numpy())

test_preds = (np.array(test_preds) > 0.5).astype(int)
f1 = f1_score(y_test, test_preds, average='macro')
print(f'Test 데이터에 대한 F1 Score: {f1:.8f}')

Test 데이터에 대한 F1 Score: 0.82513445


### Pred

In [24]:
submission = pd.read_csv(os.path.join(CONFIG.ROOT_DIR, "test_y.csv"), index_col='Serial Number')

y_test_pred = clf.predict(X_test)
submission["Y"] = y_test_pred
submission.to_csv("submission.csv", index_label='Serial Number')

FileNotFoundError: [Errno 2] No such file or directory: './data/huge\\test_y.csv'

In [ ]:
submission = pd.read_csv("submission.csv", index_col='Serial Number')
submission

In [ ]:
print(submission["Y"].mean())
print(train_y.mean())

Public Score : 0.8130563798219584